# L11: How to crunch data from Elasticsearch with python

- Elasticsearch docs: https://www.elastic.co/guide/en/elasticsearch/reference/current/index.html
- Python Elasticsearch docs: https://elasticsearch-py.readthedocs.io/en/master/

In [1]:
import pandas as pd
import requests, json
from elasticsearch import Elasticsearch

pd.set_option('display.max_columns', 150)

## Create the client and connect it to the cluster

In [2]:
host = 'http://35.184.66.234:9200'

es = Elasticsearch(host)

es.info()

{'name': '8d6c4856853c',
 'cluster_name': 'docker-cluster',
 'cluster_uuid': 'TtO4Ay54QEaJyMnLlFIlqA',
 'version': {'number': '7.0.1',
  'build_flavor': 'default',
  'build_type': 'docker',
  'build_hash': 'e4efcb5',
  'build_date': '2019-04-29T12:56:03.145736Z',
  'build_snapshot': False,
  'lucene_version': '8.0.0',
  'minimum_wire_compatibility_version': '6.7.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}

In [3]:
index_0 = 'kibana_sample_data_flights'
index_1 = 'vacancies'

## Match query example ([doc](https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-match-query.html))

In [4]:
# All flights from Tokyo Haneda

query = {
    'query': {
        'match': {
            'Origin': 'Tokyo Haneda International Airport'
        }
    }
}

In [5]:
res = es.search(index=index_0, body=query)  # search in the 'Flights' index

sources = [hit['_source'] for hit in res['hits']['hits']]  # fetch only the body of each search hit

In [6]:
df = pd.DataFrame(sources).set_index('FlightNum')

df.head()

,AvgTicketPrice,Cancelled,Carrier,Dest,DestAirportID,DestCityName,DestCountry,DestLocation,DestRegion,DestWeather,DistanceKilometers,DistanceMiles,FlightDelay,FlightDelayMin,FlightDelayType,FlightTimeHour,FlightTimeMin,Origin,OriginAirportID,OriginCityName,OriginCountry,OriginLocation,OriginRegion,OriginWeather,dayOfWeek,timestamp
FlightNum,,,,,,,,,,,,,,,,,,,,,,,,,,
M21BD4I,969.189931,False,JetBeats,Xi'an Xianyang International Airport,XIY,Xi'an,CN,"{'lat': '34.447102', 'lon': '108.751999'}",SE-BD,Clear,2823.452661,1754.412146,True,120,NAS Delay,4.352877,261.172633,Tokyo Haneda International Airport,HND,Tokyo,JP,"{'lat': '35.552299', 'lon': '139.779999'}",SE-BD,Cloudy,0,2019-11-04T20:17:26
KL2UKVD,347.372958,False,Logstash Airways,Zurich Airport,ZRH,Zurich,CH,"{'lat': '47.464699', 'lon': '8.54917'}",CH-ZH,Cloudy,9610.295156,5971.560559,False,0,No Delay,10.010724,600.643447,Tokyo Haneda International Airport,HND,Tokyo,JP,"{'lat': '35.552299', 'lon': '139.779999'}",SE-BD,Cloudy,0,2019-11-04T22:34:35
IR1FR9Z,227.668217,False,JetBeats,Tokyo Haneda International Airport,HND,Tokyo,JP,"{'lat': '35.552299', 'lon': '139.779999'}",SE-BD,Clear,0.000000,0.000000,True,270,Carrier Delay,4.500000,270.000000,Tokyo Haneda International Airport,HND,Tokyo,JP,"{'lat': '35.552299', 'lon': '139.779999'}",SE-BD,Sunny,0,2019-11-04T14:16:01
J0I2OLF,894.899226,False,Logstash Airways,Warsaw Chopin Airport,WAW,Warsaw,PL,"{'lat': '52.16569901', 'lon': '20.96710014'}",PL-MZ,Rain,8624.183915,5358.819441,False,0,No Delay,7.985355,479.121329,Tokyo Haneda International Airport,HND,Tokyo,JP,"{'lat': '35.552299', 'lon': '139.779999'}",SE-BD,Clear,0,2019-11-04T12:43:08
OP4CO7M,505.499016,False,Logstash Airways,Vienna International Airport,VIE,Vienna,AT,"{'lat': '48.11029816', 'lon': '16.56970024'}",AT-9,Thunder & Lightning,9165.064521,5694.907068,True,30,Carrier Delay,10.683405,641.004301,Tokyo Haneda International Airport,HND,Tokyo,JP,"{'lat': '35.552299', 'lon': '139.779999'}",SE-BD,Clear,0,2019-11-04T07:31:56


## Query string query example ([doc](https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-query-string-query.html))

In [7]:
# All the vacancies in Minsk or Kiev and with the word "Программист" in the name

query = {
    'query': {
        'query_string': {
            'query': 'address.city:(минск OR киев) AND name:(программист)'
        }
    }
}

In [8]:
res = es.search(index=index_1, body=query)  # search in the 'Vacancies' index

sources = [hit['_source'] for hit in res['hits']['hits']]

In [9]:
df = pd.DataFrame(sources).set_index('id')

df.head()

,address,alternate_url,apply_alternate_url,archived,area,contacts,created_at,department,employer,has_test,insider_interview,name,premium,published_at,relations,response_letter_required,response_url,salary,snippet,sort_point_distance,timestamp_ms,type,url
id,,,,,,,,,,,,,,,,,,,,,,,
34094600,"{'city': 'Киев', 'street': 'бульвар Академика ...",https://hh.ru/vacancy/34094600,https://hh.ru/applicant/vacancy_response?vacan...,False,"{'id': '115', 'name': 'Киев', 'url': 'https://...",None,2019-10-15T12:52:33+0300,None,"{'id': '33836', 'name': 'ВІДЖИ ТРЕЙД', 'url': ...",False,None,Программист 1С,False,2019-10-15T12:52:33+0300,[],False,None,None,{'requirement': 'Высшее образование. Опыт рабо...,None,1571133153000,"{'id': 'open', 'name': 'Открытая'}",https://api.hh.ru/vacancies/34094600?host=hh.ru
34094600,"{'city': 'Киев', 'street': 'бульвар Академика ...",https://hh.ru/vacancy/34094600,https://hh.ru/applicant/vacancy_response?vacan...,False,"{'id': '115', 'name': 'Киев', 'url': 'https://...",None,2019-10-15T12:52:33+0300,None,"{'id': '33836', 'name': 'ВІДЖИ ТРЕЙД', 'url': ...",False,None,Программист 1С,False,2019-10-15T12:52:33+0300,[],False,None,None,{'requirement': 'Высшее образование. Опыт рабо...,None,1571133153000,"{'id': 'open', 'name': 'Открытая'}",https://api.hh.ru/vacancies/34094600?host=hh.ru
34422421,"{'city': 'Киев', 'street': 'Бориспольская улиц...",https://hh.ru/vacancy/34422421,https://hh.ru/applicant/vacancy_response?vacan...,False,"{'id': '115', 'name': 'Киев', 'url': 'https://...","{'name': 'Нестерова Вита', 'email': 'nesterova...",2019-11-05T17:19:45+0300,None,"{'id': '7045', 'name': 'Оптима-фарм, ЛТД', 'ur...",False,None,Инженер-программист,False,2019-11-05T17:19:45+0300,[],False,None,None,{'requirement': 'Образование - полное высшее (...,None,1572963585000,"{'id': 'open', 'name': 'Открытая'}",https://api.hh.ru/vacancies/34422421?host=hh.ru
34411360,"{'city': 'Киев', 'street': 'улица Ушинского', ...",https://hh.ru/vacancy/34411360,https://hh.ru/applicant/vacancy_response?vacan...,False,"{'id': '115', 'name': 'Киев', 'url': 'https://...",None,2019-11-05T12:10:33+0300,None,"{'id': '3720430', 'name': 'СЕТРА', 'url': 'htt...",False,None,Программист РНР,False,2019-11-05T12:10:33+0300,[],False,None,"{'from': None, 'to': 20000, 'currency': 'UAH',...",{'requirement': 'Желаемый опыт работы PHP прог...,None,1572945033000,"{'id': 'open', 'name': 'Открытая'}",https://api.hh.ru/vacancies/34411360?host=hh.ru
34422421,"{'city': 'Киев', 'street': 'Бориспольская улиц...",https://hh.ru/vacancy/34422421,https://hh.ru/applicant/vacancy_response?vacan...,False,"{'id': '115', 'name': 'Киев', 'url': 'https://...","{'name': 'Нестерова Вита', 'email': 'nesterova...",2019-11-05T17:19:45+0300,None,"{'id': '7045', 'name': 'Оптима-фарм, ЛТД', 'ur...",False,None,Инженер-программист,False,2019-11-05T17:19:45+0300,[],False,None,None,{'requirement': 'Образование - полное высшее (...,None,1572963585000,"{'id': 'open', 'name': 'Открытая'}",https://api.hh.ru/vacancies/34422421?host=hh.ru


## Range query example ([doc](https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-range-query.html))

In [10]:
# All flights between 10 days ago from now on up to November, 13

query = {
    'query': {
        'range': {
            'timestamp': {
                # greater/less than (also "gte"/"lte" - greater/less than or equal available)
                # can be defined as ISO8601 timestamp or simply "now-NU", 
                # where N is an integer and U is the unit, e.g. "d" for days
                'gt': 'now-10d',
                'lte': '2019-11-13T06:00:00Z'
            }
        }
    }
}

In [11]:
res = es.search(index=index_0, body=query)  # search in the 'Flights' index

sources = [hit['_source'] for hit in res['hits']['hits']]

In [12]:
df = pd.DataFrame(sources).set_index('FlightNum')

df.head()

,AvgTicketPrice,Cancelled,Carrier,Dest,DestAirportID,DestCityName,DestCountry,DestLocation,DestRegion,DestWeather,DistanceKilometers,DistanceMiles,FlightDelay,FlightDelayMin,FlightDelayType,FlightTimeHour,FlightTimeMin,Origin,OriginAirportID,OriginCityName,OriginCountry,OriginLocation,OriginRegion,OriginWeather,dayOfWeek,timestamp
FlightNum,,,,,,,,,,,,,,,,,,,,,,,,,,
X98CCZO,882.982662,False,Logstash Airways,Venice Marco Polo Airport,VE05,Venice,IT,"{'lat': '45.505299', 'lon': '12.3519'}",IT-34,Sunny,8823.400140,5482.606665,False,0,No Delay,7.739825,464.389481,Cape Town International Airport,CPT,Cape Town,ZA,"{'lat': '-33.96480179', 'lon': '18.60169983'}",SE-BD,Clear,0,2019-11-04T18:27:00
UFK2WIZ,190.636904,False,Logstash Airways,Venice Marco Polo Airport,VE05,Venice,IT,"{'lat': '45.505299', 'lon': '12.3519'}",IT-34,Cloudy,0.000000,0.000000,False,0,No Delay,0.000000,0.000000,Venice Marco Polo Airport,VE05,Venice,IT,"{'lat': '45.505299', 'lon': '12.3519'}",IT-34,Rain,0,2019-11-04T17:11:14
EVARI8I,180.246816,False,JetBeats,Zurich Airport,ZRH,Zurich,CH,"{'lat': '47.464699', 'lon': '8.54917'}",CH-ZH,Hail,0.000000,0.000000,True,300,Security Delay,5.000000,300.000000,Zurich Airport,ZRH,Zurich,CH,"{'lat': '47.464699', 'lon': '8.54917'}",CH-ZH,Clear,0,2019-11-04T13:49:53
JQ2XXQ5,906.437948,False,JetBeats,Helsinki Vantaa Airport,HEL,Helsinki,FI,"{'lat': '60.31719971', 'lon': '24.9633007'}",FI-ES,Rain,8551.767893,5313.822211,False,0,No Delay,8.384086,503.045170,Albuquerque International Sunport Airport,ABQ,Albuquerque,US,"{'lat': '35.040199', 'lon': '-106.609001'}",US-NM,Rain,0,2019-11-04T22:06:14
VT9O2KD,374.959276,False,Logstash Airways,Ottawa Macdonald-Cartier International Airport,YOW,Ottawa,CA,"{'lat': '45.32249832', 'lon': '-75.66919708'}",CA-ON,Rain,6938.783926,4311.560441,False,0,No Delay,5.506971,330.418282,Naples International Airport,NA01,Naples,IT,"{'lat': '40.886002', 'lon': '14.2908'}",IT-72,Rain,0,2019-11-04T14:21:13


## Bool query example ([doc](https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-bool-query.html))

In [13]:
# Search for all the flight sthat match several criteria: 
# the origin city is Ottawa, the flight was longer than 10 hours and it was flying within the last 15 days from now on

query = {
    'query': {
        'bool': {
            'must': [
                {
                    'match': {
                        'OriginCityName': 'Ottawa'
                    }
                },
                {
                    'query_string': {
                        'query': 'FlightTimeHour:>=10'
                    }
                },
                {
                    'range': {
                        'timestamp': {
                            'gt': 'now-15d',
                            'lte': 'now'
                        }
                    }
                }
            ]
        }
    }
}

In [14]:
res = es.search(index=index_0, body=query)

sources = [hit['_source'] for hit in res['hits']['hits']]

In [15]:
df = pd.DataFrame(sources).set_index('FlightNum')

df.head()

,AvgTicketPrice,Cancelled,Carrier,Dest,DestAirportID,DestCityName,DestCountry,DestLocation,DestRegion,DestWeather,DistanceKilometers,DistanceMiles,FlightDelay,FlightDelayMin,FlightDelayType,FlightTimeHour,FlightTimeMin,Origin,OriginAirportID,OriginCityName,OriginCountry,OriginLocation,OriginRegion,OriginWeather,dayOfWeek,timestamp
FlightNum,,,,,,,,,,,,,,,,,,,,,,,,,,
NRE9J26,374.899128,False,Logstash Airways,Vienna International Airport,VIE,Vienna,AT,"{'lat': '48.11029816', 'lon': '16.56970024'}",AT-9,Rain,6609.346596,4106.857574,False,0,No Delay,6.479752,388.785094,Ottawa Macdonald-Cartier International Airport,YOW,Ottawa,CA,"{'lat': '45.32249832', 'lon': '-75.66919708'}",CA-ON,Clear,0,2019-11-04T06:23:10
7ZTKP49,893.927742,False,ES-Air,Pisa International Airport,PI05,Pisa,IT,"{'lat': '43.683899', 'lon': '10.3927'}",IT-52,Cloudy,6496.174118,4036.535456,False,0,No Delay,5.155694,309.341625,Ottawa Macdonald-Cartier International Airport,YOW,Ottawa,CA,"{'lat': '45.32249832', 'lon': '-75.66919708'}",CA-ON,Rain,0,2019-11-04T13:10:09
5SIPOMV,576.047399,False,Kibana Airlines,Shanghai Hongqiao International Airport,SHA,Shanghai,CN,"{'lat': '31.19790077', 'lon': '121.3359985'}",SE-BD,Cloudy,11358.349188,7057.750977,False,0,No Delay,13.521844,811.310656,Ottawa Macdonald-Cartier International Airport,YOW,Ottawa,CA,"{'lat': '45.32249832', 'lon': '-75.66919708'}",CA-ON,Heavy Fog,0,2019-11-04T15:56:09
1X3WKIL,916.421259,False,Kibana Airlines,Turin Airport,TO11,Torino,IT,"{'lat': '45.200802', 'lon': '7.64963'}",IT-21,Heavy Fog,6221.098873,3865.611623,False,0,No Delay,5.457104,327.426256,Ottawa Macdonald-Cartier International Airport,YOW,Ottawa,CA,"{'lat': '45.32249832', 'lon': '-75.66919708'}",CA-ON,Sunny,1,2019-11-05T02:40:32
UYHN6BP,664.041076,False,ES-Air,Philadelphia International Airport,PHL,Philadelphia,US,"{'lat': '39.87189865', 'lon': '-75.2410965'}",US-PA,Clear,606.495532,376.858852,True,105,Carrier Delay,2.282014,136.920817,Ottawa Macdonald-Cartier International Airport,YOW,Ottawa,CA,"{'lat': '45.32249832', 'lon': '-75.66919708'}",CA-ON,Rain,1,2019-11-05T14:04:19


## Date Histogram aggregation example ([doc](https://www.elastic.co/guide/en/elasticsearch/reference/current/search-aggregations-bucket-datehistogram-aggregation.html))

In [16]:
# How many fligths per day where delayed by more than 2 hours

query = {
    'query': {
        'query_string': {
            'query': 'FlightDelayMin:>120'
        }
    },
    'aggs': {
        'per_day': {
            'date_histogram': {
                'field': 'timestamp',
                'interval': '1d'
            }
        }
    },
    'size': 0  # 'size' is 0 in this case because we don't really need the results themselves - only the time aggregation
}

In [17]:
res = es.search(index=index_0, body=query)

buckets = res['aggregations']['per_day']['buckets']

In [18]:
df = pd.DataFrame(buckets).set_index('key_as_string')

df.head(15)

,doc_count,key
key_as_string,,
2019-11-04T00:00:00.000Z,48,1572825600000
2019-11-05T00:00:00.000Z,47,1572912000000
2019-11-06T00:00:00.000Z,62,1572998400000
2019-11-07T00:00:00.000Z,61,1573084800000
2019-11-08T00:00:00.000Z,57,1573171200000
2019-11-09T00:00:00.000Z,50,1573257600000
2019-11-10T00:00:00.000Z,41,1573344000000
2019-11-11T00:00:00.000Z,63,1573430400000
2019-11-12T00:00:00.000Z,60,1573516800000


## Elastic saved object API ([doc](https://www.elastic.co/guide/en/kibana/master/saved-objects-api.html))

In [19]:
r = requests.get('http://104.198.35.86:5601' + '/api/saved_objects/_find?type=' + 'dashboard', verify=False)
r.json()

{'page': 1,
 'per_page': 20,
 'total': 1,
 'saved_objects': [{'type': 'dashboard',
   'id': '7adfa750-4c81-11e8-b3d7-01146121b73d',
   'attributes': {'title': '[Flights] Global Flight Dashboard',
    'hits': 0,
    'description': 'Analyze mock flight data for ES-Air, Logstash Airways, Kibana Airlines and JetBeats',
    'panelsJSON': '[{"panelIndex":"1","gridData":{"x":0,"y":0,"w":32,"h":7,"i":"1"},"embeddableConfig":{},"version":"6.3.0","panelRefName":"panel_0"},{"panelIndex":"3","gridData":{"x":17,"y":7,"w":23,"h":12,"i":"3"},"embeddableConfig":{"vis":{"colors":{"Average Ticket Price":"#0A50A1","Flight Count":"#82B5D8"},"legendOpen":false}},"version":"6.3.0","panelRefName":"panel_1"},{"panelIndex":"4","gridData":{"x":0,"y":85,"w":48,"h":15,"i":"4"},"embeddableConfig":{},"version":"6.3.0","panelRefName":"panel_2"},{"panelIndex":"5","gridData":{"x":0,"y":7,"w":17,"h":12,"i":"5"},"embeddableConfig":{"vis":{"colors":{"ES-Air":"#447EBC","JetBeats":"#65C5DB","Kibana Airlines":"#BA43A9","Log

## Elastic _cat API ([doc](https://www.elastic.co/guide/en/elasticsearch/reference/current/cat.html))

In [20]:
r = requests.get(host + '/_cat/indices?format=json', verify=False)
r.json()

[{'health': 'green',
  'status': 'open',
  'index': '.kibana_task_manager',
  'uuid': 'KyMD_3bxRryirGbkip9KWQ',
  'pri': '1',
  'rep': '0',
  'docs.count': '2',
  'docs.deleted': '0',
  'store.size': '31kb',
  'pri.store.size': '31kb'},
 {'health': 'yellow',
  'status': 'open',
  'index': 'api',
  'uuid': 's7S3jLdXSo2u8J4ENkT2-w',
  'pri': '1',
  'rep': '1',
  'docs.count': '1',
  'docs.deleted': '0',
  'store.size': '2.6kb',
  'pri.store.size': '2.6kb'},
 {'health': 'green',
  'status': 'open',
  'index': 'kibana_sample_data_flights',
  'uuid': '3wSM3M4qSTWy0B9LB37Ktg',
  'pri': '1',
  'rep': '0',
  'docs.count': '13059',
  'docs.deleted': '0',
  'store.size': '6.4mb',
  'pri.store.size': '6.4mb'},
 {'health': 'yellow',
  'status': 'open',
  'index': 'vacancies',
  'uuid': 'oxG2z2nxTXWo7-2anAZE8g',
  'pri': '1',
  'rep': '1',
  'docs.count': '10436',
  'docs.deleted': '0',
  'store.size': '18.9mb',
  'pri.store.size': '18.9mb'},
 {'health': 'green',
  'status': 'open',
  'index': '.ki

In [21]:
r = requests.get(host + '/_cluster/health', verify=False)
r.json()

{'cluster_name': 'docker-cluster',
 'status': 'yellow',
 'timed_out': False,
 'number_of_nodes': 1,
 'number_of_data_nodes': 1,
 'active_primary_shards': 5,
 'active_shards': 5,
 'relocating_shards': 0,
 'initializing_shards': 0,
 'unassigned_shards': 2,
 'delayed_unassigned_shards': 0,
 'number_of_pending_tasks': 0,
 'number_of_in_flight_fetch': 0,
 'task_max_waiting_in_queue_millis': 0,
 'active_shards_percent_as_number': 71.42857142857143}

In [22]:
r = requests.get(host + '/_nodes/stats', verify=False)
r.json()

{'_nodes': {'total': 1, 'successful': 1, 'failed': 0},
 'cluster_name': 'docker-cluster',
 'nodes': {'f3PAEZmFRzaMUjVTBOrR-Q': {'timestamp': 1573651108999,
   'name': '8d6c4856853c',
   'transport_address': '172.18.0.2:9300',
   'host': '172.18.0.2',
   'ip': '172.18.0.2:9300',
   'roles': ['master', 'data', 'ingest'],
   'attributes': {'ml.machine_memory': '63321321472',
    'xpack.installed': 'true',
    'ml.max_open_jobs': '20'},
   'indices': {'docs': {'count': 23569, 'deleted': 10},
    'store': {'size_in_bytes': 26836917},
    'indexing': {'index_total': 34098,
     'index_time_in_millis': 7487,
     'index_current': 0,
     'index_failed': 1,
     'delete_total': 1,
     'delete_time_in_millis': 8,
     'delete_current': 0,
     'noop_update_total': 0,
     'is_throttled': False,
     'throttle_time_in_millis': 0},
    'get': {'total': 18875,
     'time_in_millis': 2421,
     'exists_total': 18816,
     'exists_time_in_millis': 2400,
     'missing_total': 59,
     'missing_time_